# Notebook para crear la tabla pedidos

In [2]:
#Funciones y librerías
import psycopg2
import pandas as pd
import numpy as np
import datetime

In [3]:
def data_report(pedidos):
    
    '''
    Dscribe los campos de un pedidos de pandas de forma clara, dando el tipo de columna, 
    el porcentaje de Missings, la cantidad de valores únicos y el porcentaje de valores únicos respecto 
    al total de valores de esa variable (Cardin)
    Si no hay columnas categóricas, además muestra el valor máximo, el mínimo y el valor medio.
    '''
    # Librerias
    import pandas as pd
    
    # Se saca el nombre de las columnas
    cols = pd.DataFrame(pedidos.columns.values, columns=["COL_N"])

    # Se saca el tipo de los datos
    types = pd.DataFrame(pedidos.dtypes.values, columns=["Data Type"])

    # Se sacan los Missings
    percent_missing = round(pedidos.isnull().sum() * 100 / len(pedidos), 2)
    percent_missing_pedidos = pd.DataFrame(percent_missing.values, columns=["Missings (%)"])

    # Se sacan los Valores únicos
    unicos = pd.DataFrame(pedidos.nunique().values, columns=["Unique Values"])
    percent_cardin = round(unicos['Unique Values']*100/len(pedidos), 2)
    percent_cardin_pedidos = pd.DataFrame(percent_cardin.values, columns=["CARDIN (%)"])
    
    #Se concatenan todos los campos
    concatenado = pd.concat([cols, types, percent_missing_pedidos, unicos, percent_cardin_pedidos], axis=1, sort=False)
    concatenado.set_index('COL_N', drop=True, inplace=True)

    return concatenado.T #el T es un transpon del pedidos

Primero conexión a la BBDD para obtener las tablas

In [3]:
SECRET_KEY = "PA4SSW0R1*_?"
PGSQL_HOST = "appreparto-db.c0ttdmuj4kco.eu-north-1.rds.amazonaws.com"
PGSQL_USER = "durumkebab"
PGSQL_PASSWORD = "Qu3l3ch35353l5h4w4rm4!"
PGSQL_DATABASE = "repartos_test"

# Establecer la conexión con la base de datos
conn = psycopg2.connect(
    host=PGSQL_HOST,
    database=PGSQL_DATABASE,
    user=PGSQL_USER,
    password=PGSQL_PASSWORD,
    sslkey=SECRET_KEY
)

# Crear un cursor
cur = conn.cursor()

# Ejecutar una consulta
cur.execute("SELECT * FROM clientes")

# Recuperar los resultados y guardarlos en un dataframe
clientes = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

# Ejecutar una consulta
cur.execute("SELECT * FROM comercios")

# Recuperar los resultados y guardarlos en un dataframe
comercios = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

# Ejecutar una consulta
cur.execute("SELECT * FROM repartidores")

# Recuperar los resultados y guardarlos en un dataframe
repartidores = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

# Cerrar el cursor y la conexión
cur.close()
conn.close()


In [4]:
clientes

,id,nombre,email,direccion,latitud,longitud
0,1,Cristóbal Ortega Pozo,cristobal.ortega.pozo@mail.com,"Calle de Torrelaguna, 16, Madrid",40.441461,-3.650346
1,2,Eloy Riera-Herrero,eloy.riera-herrero@mail.com,"Calle de Jaime I El Conquistador, 35, Madrid",40.394304,-3.698146
2,3,Chus Llorente Carbajo,chus.llorente.carbajo@mail.com,"Calle Santa Amalia, 5, Madrid",40.401937,-3.663755
3,4,Dolores Guerra Matas,dolores.guerra.matas@mail.com,"Calle del Ángel, 15, Madrid",40.409845,-3.712690
4,5,Laura del Suarez,laura.del.suarez@mail.com,"Calle de Caramuel, 51, Madrid",40.409063,-3.728480
...,...,...,...,...,...,...
9997,9999,Mónica Gomis Acevedo,monica.gomis.acevedo@mail.com,"Calle de La del Manojo de Rosas, 20, Madrid",40.359189,-3.698213
9998,10000,Carmina Girón,carmina.giron@mail.com,"Plaza Castillo de la Mota, 5, Madrid",40.427173,-3.628923
9999,10001,Máxima de Barbero,maxima.de.barbero@mail.com,"Calle de Antonio Leyva, 53, Madrid",40.390470,-3.715673
10000,10002,Hombre De Incognito,incognito@mail.com,"Calle Falsa, 123, Madrid",40.436000,40.436000


In [6]:
comercios

,id,nombre,zona,direccion,tipo,latitud,longitud
0,1,Astor,28013,"Plaza de la Puerta del Sol, 44, Madrid",armenian,40.416760,-3.703850
1,2,Restaurante Angel Mira,28013,"Plaza de la Puerta del Sol, 58, Madrid",armenian,40.416760,-3.703850
2,3,Cafetería Armenia,28013,"Calle del Carmen, 3, Madrid",armenian,40.417660,-3.703710
3,4,Oroya,28013,"Plaza del Celenque, 2, Madrid",armenian,40.417350,-3.705960
4,5,Pe de Pisco,28013,"Calle de la Aduana, 5, Madrid",armenian,40.418160,-3.702040
...,...,...,...,...,...,...,...
1128,1129,Reverso Madrid,28042,"Avenida de Los Andes, 8, Madrid",steak house,40.453770,-3.624160
1129,1130,La Casa de Las Brasas,28043,"Calle Tribaldos, 38, Madrid",steak house,40.466860,-3.636060
1130,1131,Casa Justo,28224,"Avenida de Europa, 11, Pozuelo de Alarcón",steak house,40.437940,-3.794130
1131,1132,Restaurante Conduma-Abierto-Abierto,28033,"Calle del Mar de Bering, 7, Madrid",steak house,40.473000,-3.641790


In [7]:
repartidores

,id,nombre,email,status,vehiculo,ocupado,zona,latitud,longitud,datetime_ult_act
0,2,Lucas Salgado Leiva,lucas.salgado.leiva@mail.com,False,Motocicleta,None,None,None,None,None
1,3,Valentina Pol Estrada,valentina.pol.estrada@mail.com,False,Patín,None,None,None,None,None
2,4,Gastón Varela,gaston.varela@mail.com,False,Motocicleta,None,None,None,None,None
3,5,Guadalupe Iglesia Sarmiento,guadalupe.iglesia.sarmiento@mail.com,False,Bicicleta,None,None,None,None,None
4,6,Milagros Adán Calatayud,milagros.adan.calatayud@mail.com,False,Motocicleta,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
495,497,Sigfrido Castrillo,sigfrido.castrillo@mail.com,False,Patín,None,None,None,None,None
496,498,Rosa María Marín Tomas,rosa.maria.marin.tomas@mail.com,False,Motocicleta,None,None,None,None,None
497,499,Quirino Polo Torres,quirino.polo.torres@mail.com,False,Bicicleta,None,None,None,None,None
498,500,Montserrat Lara Pino,montserrat.lara.pino@mail.com,False,Bicicleta,None,None,None,None,None


# CSV de pedidos de Kaggle

In [8]:
pedidos_kaggle_1 = pd.read_csv('./Datos/Pedidos Kaggle/train.csv')
pedidos_kaggle_2 = pd.read_csv('./Datos/Pedidos Kaggle/test.csv')

# concatenando los dos dataframes por filas
pedidos_kaggle = pd.concat([pedidos_kaggle_1, pedidos_kaggle_2])


In [9]:
pedidos_kaggle

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11394,0x6909,JAPRES01DEL01,35,4.6,26.905190,75.810753,27.015190,75.920753,27-03-2022,21:35:00,21:45:00,conditions Sunny,Jam,1,Snack,scooter,1,No,Metropolitian,NaN
11395,0x443b,JAPRES11DEL01,33,4.9,26.902940,75.793007,26.912940,75.803007,11-03-2022,11:40:00,11:45:00,conditions Sandstorms,High,1,Drinks,scooter,1,No,Metropolitian,NaN
11396,0x1ea5,SURRES11DEL03,NaN,NaN,21.157735,72.768778,21.217735,72.828778,11-03-2022,NaN,21:05:00,conditions NaN,NaN,3,Drinks,scooter,1,No,Metropolitian,NaN
11397,0x22d4,VADRES03DEL02,27,4.7,22.320000,73.170000,22.450000,73.300000,06-03-2022,18:35:00,18:40:00,conditions Sandstorms,Medium,0,Meal,motorcycle,0,No,Metropolitian,NaN


Seleccionamos solo las columnas que interesan

In [10]:
pedidos_kaggle.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)'],
      dtype='object')

In [11]:
pedidos = pedidos_kaggle.loc[:, ["ID", "Order_Date", "Time_Orderd", "Time_Order_picked", "Weatherconditions", "Road_traffic_density", "Type_of_vehicle", "Festival"]]

# Se renombra la columna 'ID' como 'ID_Pedidos'
pedidos = pedidos.rename(columns={'ID': 'ID_Pedidos'})

pedidos

,ID_Pedidos,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Type_of_vehicle,Festival
0,0x4607,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,motorcycle,No
1,0xb379,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,scooter,No
2,0x5d6d,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,motorcycle,No
3,0x7a6a,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,motorcycle,No
4,0x70a2,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,scooter,No
...,...,...,...,...,...,...,...,...
11394,0x6909,27-03-2022,21:35:00,21:45:00,conditions Sunny,Jam,scooter,No
11395,0x443b,11-03-2022,11:40:00,11:45:00,conditions Sandstorms,High,scooter,No
11396,0x1ea5,11-03-2022,NaN,21:05:00,conditions NaN,NaN,scooter,No
11397,0x22d4,06-03-2022,18:35:00,18:40:00,conditions Sandstorms,Medium,motorcycle,No


# Data Cleaning

In [12]:
data_report(pedidos)

COL_N,ID_Pedidos,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Type_of_vehicle,Festival
Data Type,object,object,object,object,object,object,object,object
Missings (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Unique Values,56992,44,177,193,7,5,4,3
CARDIN (%),100.0,0.08,0.31,0.34,0.01,0.01,0.01,0.01


In [13]:
# eliminar las filas que contienen NaN
pedidos = pedidos.dropna()
pedidos = pedidos.drop(pedidos[(pedidos == 'NaN').any(axis=1)].index)
pedidos = pedidos.drop(pedidos[(pedidos == 'NaN ').any(axis=1)].index)

In [14]:
# limpiar columna de tiempo
pedidos['Weatherconditions']=pedidos['Weatherconditions'].str.split(" ", expand=True)[1]

In [15]:
# Convertir la columna Order_Date
pedidos['Order_Date']=pd.to_datetime(pedidos['Order_Date'],format="%d-%m-%Y")

In [16]:
# Convertir las columnas temporales
pedidos['Time_Orderd']=pd.to_datetime(pedidos['Time_Orderd'], format="%H:%M:%S").dt.time
pedidos['Time_Order_picked']=pd.to_datetime(pedidos['Time_Order_picked'], format="%H:%M:%S").dt.time

In [17]:
# Se crean dos columnas de hora temporales
pedidos['Time_Orderd_temp'] = pd.to_datetime(pedidos['Time_Orderd'], format="%H:%M:%S")
pedidos['Time_Order_picked_temp'] = pd.to_datetime(pedidos['Time_Order_picked'], format="%H:%M:%S")

# Calcular la diferencia entre las dos columnas de hora
pedidos['duracion(min)'] = (pedidos['Time_Order_picked_temp'] - pedidos['Time_Orderd_temp']).dt.seconds.div(60).astype(int)

#Se eliminan las columnas que no sirven
pedidos = pedidos.drop(['Time_Orderd_temp', "Time_Order_picked_temp"], axis=1)

In [18]:
pedidos

,ID_Pedidos,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Type_of_vehicle,Festival,duracion(min)
1,0xb379,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,No,5
2,0x5d6d,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,No,15
3,0x7a6a,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,No,10
4,0x70a2,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,No,15
5,0x9bb4,2022-03-11,21:20:00,21:30:00,Cloudy,Jam,motorcycle,No,10
...,...,...,...,...,...,...,...,...,...
11393,0xe240,2022-02-13,11:00:00,11:10:00,Windy,Low,scooter,No,10
11394,0x6909,2022-03-27,21:35:00,21:45:00,Sunny,Jam,scooter,No,10
11395,0x443b,2022-03-11,11:40:00,11:45:00,Sandstorms,High,scooter,No,5
11397,0x22d4,2022-03-06,18:35:00,18:40:00,Sandstorms,Medium,motorcycle,No,5


# Inclusión de los ID de clientes, comercios y repartidores

In [19]:
# Se genera la columna de ID de clientes con valores entre 1 y 1002
pedidos['ID_Clientes'] = np.random.randint(1, clientes['id'].max()+1, size=len(pedidos))

# Se genera la columna de ID de comercios con valores entre 1 y 188
pedidos['ID_Comercios'] = np.random.randint(1, comercios['id'].max()+1, size=len(pedidos))

# Se genera la columna de ID de repartidores con valores entre 1 y 100
pedidos['ID_Repartidores'] = np.random.randint(1, repartidores['id'].max()+1, size=len(pedidos))

In [20]:
pedidos

,ID_Pedidos,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Type_of_vehicle,Festival,duracion(min),ID_Clientes,ID_Comercios,ID_Repartidores
1,0xb379,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,No,5,9560,930,226
2,0x5d6d,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,No,15,9879,997,98
3,0x7a6a,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,No,10,8616,65,128
4,0x70a2,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,No,15,3697,81,128
5,0x9bb4,2022-03-11,21:20:00,21:30:00,Cloudy,Jam,motorcycle,No,10,9715,152,460
...,...,...,...,...,...,...,...,...,...,...,...,...
11393,0xe240,2022-02-13,11:00:00,11:10:00,Windy,Low,scooter,No,10,2041,611,477
11394,0x6909,2022-03-27,21:35:00,21:45:00,Sunny,Jam,scooter,No,10,7332,738,328
11395,0x443b,2022-03-11,11:40:00,11:45:00,Sandstorms,High,scooter,No,5,5263,347,218
11397,0x22d4,2022-03-06,18:35:00,18:40:00,Sandstorms,Medium,motorcycle,No,5,9664,243,278


In [21]:
pedidos.Road_traffic_density.unique()

array(['Jam ', 'Low ', 'Medium ', 'High '], dtype=object)

In [22]:
# Exporta el DataFrame a un archivo CSV
pedidos.to_csv('pedidos_tripulaciones.csv', index=False)  